### 네이버 데이터랩 급상승 검색어 가져오기

* url 정보 확인
* 버튼에 대한 xpath 확인
* 버튼을 통해 페이지 변경
* 각 페이지별 키워드 획득
* 획득한 키워드를 xlsx, csv 파일로 저장

In [3]:
filename = 'datalab_20200527.xlsx'
sheetname = input('시트명 입력 :')

시트명 입력 :keyword


#### webdriver 이용해서 Chrome 브라우저를 제어

In [83]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [84]:
driver = webdriver.Chrome('chromedriver')

In [85]:
url = 'https://datalab.naver.com/keyword/realtimeList.naver'
driver.get(url)

In [86]:
grade_btn = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div[2]/div/div/div/ul/li[1]/a')
grade_btn.click()

#### 방법1

In [87]:
soup = BeautifulSoup(driver.page_source,'lxml')   # 드라이버의 현재 페이지 정보를 가져옴

In [88]:
all_keyword = soup.find_all('span', class_='item_title')

pop_word = []
for one in all_keyword:
    word = one.text
    pop_word.append(word)
pop_word

['이동건',
 '조윤희',
 '이동건 조윤희',
 'msc',
 '세븐틴',
 '롤 msc',
 '정일훈',
 '수상한 그녀',
 '숨바꼭질',
 '로이킴',
 '이동건 이혼',
 '퀸와사비',
 '쿠팡 고양물류센터',
 '논현2동',
 '영등포구청',
 '유베이스',
 '김지혜',
 '이영아',
 '홍콩 국가보안법',
 '양진호']

In [89]:
all_word = []
for i in range(1,7):
    grade_btn = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div[2]/div/div/div/ul/li[{}]/a'.format(i))
    grade_btn.click()   # 연령대별 페이지 변경
    
    soup = BeautifulSoup(driver.page_source,'lxml')   # 연령대별 html 정보 가져오기
    age_data = soup.find_all('span',class_='item_title')   # 해당 데이터 가져오기
    
    pop_word = []
    for one in age_data:
        text = one.text
        pop_word.append(text)
    all_word.append(pop_word)

In [90]:
import pandas as pd

In [91]:
dict_dat = {'10대':all_word[0],
            '20대':all_word[1],
            '30대':all_word[2],
            '40대':all_word[3],
            '50대':all_word[4],
            '전체':all_word[5]}
dat = pd.DataFrame(dict_dat)
dat

,10대,20대,30대,40대,50대,전체
0,이동건,이동건,이동건,이동건,조윤희,이동건
1,조윤희,조윤희,조윤희,조윤희,이동건,조윤희
2,이동건 조윤희,이동건 조윤희,이동건 조윤희,이동건 조윤희,이동건 조윤희,이동건 조윤희
3,msc,코레일 채용,이동건 이혼,이동건 이혼,이동건 이혼,이동건 이혼
4,세븐틴,msc,강서은,홍콩 국가보안법,홍콩 국가보안법,코레일 채용
5,롤 msc,롤 msc,홍콩 국가보안법,강서은,강서은,홍콩 국가보안법
6,정일훈,강서은,양진호,양진호,쿠팡 고양물류센터,강서은
7,수상한 그녀,코레일,손원락,쿠팡 고양물류센터,양진호,msc
8,숨바꼭질,이동건 이혼,빅텍,손원락,백선엽,양진호
9,로이킴,수상한 그녀,이영아,영등포구청,한국은행 기준금리,쿠팡 고양물류센터


In [82]:
dat.to_excel(filename, sheet_name=sheetname, index=False)

#### 방법2

In [62]:
dat = pd.DataFrame()

for i in range(1,6):    
    grade_btn = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div[2]/div/div/div/ul/li[{}]/a'.format(i))
    grade_btn.click()

    keywords = []
    for a in range(1,3):
        for b in range(1,11):
            key = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[2]/div[2]/div/div/ul[{}]/li[{}]/div/span[2]/span'.format(a,b)).text
            keywords.append(key)
            
    g = pd.DataFrame({'keyword':keywords})
    g.rename(columns = {'keyword' : '{}대'.format(i*10)}, inplace = True)
    dat = pd.concat([dat,g], axis=1)


In [69]:
dat.index = range(1,21)

In [70]:
dat

,10대,20대,30대,40대,50대
1,이동건,이동건,이동건,이동건,이동건
2,조윤희,조윤희,조윤희 이동건,조윤희 이동건,조윤희 이동건
3,조윤희 이동건,조윤희 이동건,조윤희,조윤희,조윤희
4,이동건 조윤희,이동건 조윤희,이동건 조윤희,한국은행 기준금리,한국은행 기준금리
5,로이킴,로이킴,기준금리,이동건 조윤희,팽현숙 카페
6,쿠팡 확진자,기준금리,이동건 이혼,쿠팡 확진자,송철호
7,허윤동,쿠팡 확진자,어쩌다 결혼,어쩌다 결혼,코디엠
8,부천,이동건 이혼,쿠팡 확진자,이동건 이혼,어쩌다 결혼
9,이동건 이혼,어쩌다 결혼,남선알미늄,홍콩 국가보안법,쿠팡 확진자
10,고양 쿠팡,김강열,약쿠르트,권순관,홍콩 국가보안법


In [72]:
dat.to_excel('연령대별 급상승검색어.xlsx')